# Aligners

> Collection of Aligner models

In [ ]:
#| default_exp models.aligners

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Wav2Vec2.0 Aligner

In [ ]:
#| export
from nimrod.text.normalizers import TTSTextNormalizer
from nimrod.text.normalizers import Punctuation
import torch
import torchaudio
from dataclasses import dataclass

In [ ]:
#| export

@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5d}, {self.end:5d})"

    @property
    def length(self):
        return self.end - self.start

@dataclass
class Point:
    token_index: int
    time_index: int
    score: float


class AlignerWAV2VEC2():
    def __init__(self, text_normalizer, device='cuda'):
        self._asr = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
        self.model = self._asr.get_model().to(device)
        self.labels = self._asr.get_labels()
        self._dictionary = {c: i for i, c in enumerate(self.labels)}
        self.device = device
        self.text_normalizer = text_normalizer
    
    @property
    def dictionary(self):
        return self._dictionary

    @property
    def asr(self):
        return self._asr

    def get_emission(self, waveform):
        with torch.inference_mode():
            # waveform, _ = torchaudio.load(speech_file)
            emissions, _ = self.model(waveform.to(self.device))
            emissions = torch.log_softmax(emissions, dim=-1)
        emission = emissions[0].cpu().detach()
        return emission, waveform

    def get_trellis(self, emission, tokens, blank_id=0):
        num_frame = emission.size(0)
        num_tokens = len(tokens)
        trellis = torch.empty((num_frame + 1, num_tokens + 1))
        trellis[0, 0] = 0
        trellis[1:, 0] = torch.cumsum(emission[:, 0], 0)
        trellis[0, -num_tokens:] = -float("inf")
        trellis[-num_tokens:, 0] = float("inf")

        for t in range(num_frame):
            trellis[t + 1, 1:] = torch.maximum(
                # Score for staying at the same token
                trellis[t, 1:] + emission[t, blank_id],
                # Score for changing to the next token
                trellis[t, :-1] + emission[t, tokens],
            )
        return trellis

    def backtrack(self, trellis, emission, tokens, blank_id=0):
        j = trellis.size(1) - 1
        t_start = torch.argmax(trellis[:, j]).item()

        path = []
        for t in range(t_start, 0, -1):
            stayed = trellis[t - 1, j] + emission[t - 1, blank_id]
            changed = trellis[t - 1, j - 1] + emission[t - 1, tokens[j - 1]]
            prob = emission[t - 1, tokens[j - 1] if changed > stayed else 0].exp().item()
            path.append(Point(j - 1, t - 1, prob))
            if changed > stayed:
                j -= 1
                if j == 0:
                    break
        else:
            raise ValueError("Failed to align")
        return path[::-1]

    def merge_repeats(self, path, transcript):
        i1, i2 = 0, 0
        segments = []
        while i1 < len(path):
            while i2 < len(path) and path[i1].token_index == path[i2].token_index:
                i2 += 1
            score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
            segments.append(
                Segment(
                    transcript[path[i1].token_index],
                    path[i1].time_index,
                    path[i2 - 1].time_index + 1,
                    score,
                )
            )
            i1 = i2
        return segments

    def merge_words(self, segments, separator="|"):
        words = []
        i1, i2 = 0, 0
        while i1 < len(segments):
            if i2 >= len(segments) or segments[i2].label == separator:
                if i1 != i2:
                    segs = segments[i1:i2]
                    word = "".join([seg.label for seg in segs])
                    score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                    words.append(Segment(word, segments[i1].start, segments[i2 - 1].end, score))
                i1 = i2 + 1
                i2 = i1
            else:
                i2 += 1
        return words
    
    def text_processing(self, text:str):
        cleaned = self.text_normalizer(text)
        punc = Punctuation()
        depuncted = punc.strip(cleaned)
        return depuncted.upper().replace(" ", "|")

    def get_alignments(self, speech_waveform, transcript):
        # with open(transcript_file, "r") as f:
        #     transcript = f.read()
        transcript = self.text_processing(transcript)
        emission, waveform = self.get_emission(speech_waveform)
        tokens = [self.dictionary[c] for c in transcript]
        trellis = self.get_trellis(emission, tokens)
        path = self.backtrack(trellis, emission, tokens)
        segments = self.merge_repeats(path, transcript)
        word_segments = self.merge_words(segments)
        alignments = []
        for i in range(len(word_segments)):
            ratio = waveform.size(1) / (trellis.size(0) - 1)
            word = word_segments[i]
            x0 = int(ratio * word.start)
            x1 = int(ratio * word.end)
            start = x0 / self.asr.sample_rate
            end = x1 / self.asr.sample_rate
            alignments.append(
                {"word": word.label, "score": word.score, "start_time": start, "end_time": end}
            )
            print(f"Word: {word.label}, Confidence: {word.score:.2f}, Start:{x0 / self.asr.sample_rate:.3f},  End: {x1 / self.asr.sample_rate:.3f} sec")
        return(alignments)

## Usage

In [ ]:
text_normalizer = TTSTextNormalizer().english_cleaners
aligner = AlignerWAV2VEC2(text_normalizer, device='cpu') # for CI on cpu
wav_path = "../data/en/LibriTTS/test-clean/1089/134686/1089_134686_000015_000001.wav"
txt_path = "../data/en/LibriTTS/test-clean/1089/134686/1089_134686_000015_000001.original.txt"
wav, sr = torchaudio.load(wav_path)
with open(txt_path, 'r') as f: txt = f.read()
alignments = aligner.get_alignments(wav, txt)

Word: HE, Confidence: 1.00, Start:0.121,  End: 0.242 sec
Word: TRIED, Confidence: 0.91, Start:0.323,  End: 0.625 sec
Word: TO, Confidence: 1.00, Start:0.686,  End: 0.787 sec
Word: THINK, Confidence: 0.93, Start:0.948,  End: 1.311 sec
Word: HOW, Confidence: 0.91, Start:1.473,  End: 1.675 sec
Word: IT, Confidence: 0.71, Start:1.755,  End: 1.836 sec
Word: COULD, Confidence: 0.75, Start:1.917,  End: 2.118 sec
Word: BE, Confidence: 1.00, Start:2.239,  End: 2.461 sec


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()